In [87]:
import random
# utilizado para la manipulación de directorios y rutas
import os

# Cálculo científico y vectorial para python
import numpy as np

# Libreria para graficos
from matplotlib import pyplot

# Modulo de optimizacion en scipy
from scipy import optimize

# le dice a matplotlib que incruste gráficos en el cuaderno
%matplotlib inline

In [88]:
class MLP:
    def __init__(self, layers):
        # el MLP es una lista de capas
        self.layers = layers

    def __call__(self, x):
        # calculamos la salida del modelo aplicando
        # cada capa de manera secuencial
        for layer in self.layers:
            x = layer(x)
        return x

In [89]:
class Layer():
    def __init__(self):
        self.params = []
        self.grads = []

    def __call__(self, x):
        # por defecto, devolver los inputs
        # cada capa hará algo diferente aquí
        return x

    def backward(self, grad):
        # cada capa, calculará sus gradientes
        # y los devolverá para las capas siguientes
        return grad

    def update(self, params):
        # si hay parámetros, los actualizaremos
        # con lo que nos de el optimizer
        return

In [90]:
class Linear(Layer):
    def __init__(self, d_in, d_out):
        # pesos de la capa
        self.w = np.random.normal(loc=0.0,
        scale=np.sqrt(2/(d_in+d_out)),
        size=(d_in, d_out))
        self.b = np.zeros(d_out)

    def __call__(self, x):
        self.x = x
        self.params = [self.w, self.b]
        # salida del preceptrón
        return np.dot(x, self.w) + self.b

    def backward(self, grad_output):
        # gradientes para la capa siguiente (BACKPROP)
        grad = np.dot(grad_output, self.w.T)
        self.grad_w = np.dot(self.x.T, grad_output)
        # gradientes para actualizar pesos
        self.grad_b = grad_output.mean(axis=0)*self.x.shape[0]
        self.grads = [self.grad_w, self.grad_b]
        return grad

    def update(self, params):
        self.w, self.b = params

In [91]:
#funciones de activacion
class ReLU(Layer):
    def __call__(self, x):
        self.x = x
        return np.maximum(0, x)

    def backward(self, grad_output):
        grad = self.x > 0
        return grad_output*grad

def sigmoid(x):
  return 1 / (1 + np.exp(-x))

def softmax(x):
    return np.exp(x) / np.exp(x).sum(axis=-1,keepdims=True)

class Sigmoid(Layer):
    def __call__(self, x):
        self.x = x
        return sigmoid(x)

    def backward(self, grad_output):
        grad = sigmoid(self.x)*(1 - sigmoid(self.x))
        return grad_output*grad

In [92]:
#Optimizador, stocastic greadent descent
class SGD():
    def __init__(self, net, lr):
        self.net = net
        self.lr = lr

    def update(self):
        for layer in self.net.layers:
            layer.update([
                params - self.lr*grads
                for params, grads in zip(layer.params, layer.grads)
            ])

In [93]:
class Loss():
    def __init__(self, net):
        self.net = net

    def backward(self):
        # derivada de la loss function con respecto
        # a la salida del MLP
        grad = self.grad_loss()
        # BACKPROPAGATION
        for layer in reversed(self.net.layers):
            grad = layer.backward(grad)

class MSE(Loss):
    def __call__(self, output, target):
        self.output, self.target = output, target.reshape(output.shape)
        loss = np.mean((self.output - self.target)**2)
        return loss.mean()

    def grad_loss(self):
        return self.output -  self.target

class BCE(Loss):
    def __call__(self, output, target):
        self.output, self.target = output, target.reshape(output.shape)
        loss = - np.mean(self.target*np.log(self.output) - (1 - self.target)*np.log(1 - self.output))
        return loss.mean()

    def grad_loss(self):
        return self.output -  self.target

class CrossEntropy(Loss):
    def __call__(self, output, target):
        self.output, self.target = output, target
        logits = output[np.arange(len(output)), target]
        loss = - logits + np.log(np.sum(np.exp(output), axis=-1))
        loss = loss.mean()
        return loss

    def grad_loss(self):
        answers = np.zeros_like(self.output)
        answers[np.arange(len(self.output)), self.target] = 1
        return (- answers + softmax(self.output)) / self.output.shape[0]

In [94]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from datetime import datetime

#Carga de dataset
data = pd.read_csv('Brain Tumor.csv')

In [95]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3762 entries, 0 to 3761
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Image               3762 non-null   object 
 1   Class               3762 non-null   int64  
 2   Mean                3762 non-null   float64
 3   Variance            3762 non-null   float64
 4   Standard Deviation  3762 non-null   float64
 5   Entropy             3762 non-null   float64
 6   Skewness            3762 non-null   float64
 7   Kurtosis            3762 non-null   float64
 8   Contrast            3762 non-null   float64
 9   Energy              3762 non-null   float64
 10  ASM                 3762 non-null   float64
 11  Homogeneity         3762 non-null   float64
 12  Dissimilarity       3762 non-null   float64
 13  Correlation         3762 non-null   float64
 14  Coarseness          3762 non-null   float64
dtypes: float64(13), int64(1), object(1)
memory usage: 441.0

In [96]:
columnas_eiliminar = ['Image']
data = data.drop(columnas_eiliminar, axis=1)

In [97]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3762 entries, 0 to 3761
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Class               3762 non-null   int64  
 1   Mean                3762 non-null   float64
 2   Variance            3762 non-null   float64
 3   Standard Deviation  3762 non-null   float64
 4   Entropy             3762 non-null   float64
 5   Skewness            3762 non-null   float64
 6   Kurtosis            3762 non-null   float64
 7   Contrast            3762 non-null   float64
 8   Energy              3762 non-null   float64
 9   ASM                 3762 non-null   float64
 10  Homogeneity         3762 non-null   float64
 11  Dissimilarity       3762 non-null   float64
 12  Correlation         3762 non-null   float64
 13  Coarseness          3762 non-null   float64
dtypes: float64(13), int64(1)
memory usage: 411.6 KB


In [98]:
x = data.iloc[:, 0:]
y = data.iloc[:, 0]
m = y.size
print(x.shape)
print(y.shape)
print(x)
print(y)

(3762, 14)
(3762,)
      Class       Mean     Variance  Standard Deviation   Entropy  Skewness  \
0         0   6.535339   619.587845           24.891522  0.109059  4.276477   
1         0   8.749969   805.957634           28.389393  0.266538  3.718116   
2         1   7.341095  1143.808219           33.820234  0.001467  5.061750   
3         1   5.958145   959.711985           30.979219  0.001477  5.677977   
4         0   7.315231   729.540579           27.010009  0.146761  4.283221   
...     ...        ...          ...                 ...       ...       ...   
3757      0  21.234512  1208.850174           34.768523  0.063774  2.082079   
3758      0  20.435349  1227.151440           35.030721  0.066763  2.144625   
3759      0  18.011520  1151.582765           33.934978  0.068396  2.308349   
3760      0  13.330429   945.732779           30.752769  0.087872  2.732822   
3761      0   6.110138   480.884025           21.929068  0.118171  4.110669   

       Kurtosis    Contrast    E

In [99]:
# Crea un nuevo DataFrame con los datos modificados
nuevo_data = data.copy()
# Guardar el dataset actualizado en un nuevo archivo
nuevo_data.to_csv('tumor.csv', index=False)

In [100]:
data = np.loadtxt("tumor.csv", delimiter=',',skiprows=1)
# print(data)
x, y = data[:, :13].astype(int), data[:, 0].astype(int)
x = x.reshape(len(x),13)
print(x.shape)
print(y.shape)

(3762, 13)
(3762,)


In [101]:
# #Normalización entre 0 y 1

X_mean, X_std = x.mean(axis=0), x.std(axis=0)
X_norm = (x - X_mean) / X_std


C:\Users\hlope\AppData\Local\Temp\ipykernel_11668\2052036624.py:4: RuntimeWarning: invalid value encountered in divide
  X_norm = (x - X_mean) / X_std


In [102]:
x_train, x_test, y_train, y_test = x[:1881] , x[1879:] , y[:1881].astype(np.int64), y[1879:].astype(np.int64)
x_train.shape , x_test.shape

y_train.shape , y_test.shape

((1881,), (1883,))

In [103]:
# Funcion de activacion para la capa oculta del mlp
def relu(x):
  return np.maximum(0, x)

def reluPrime(x):
  return x > 0

In [104]:
def sigmoid(x):
  return 1 / (1 + np.exp(-x))

In [105]:
# Binary Cross Entropy -> usada para clasificación binaria (con sigmoid)
def bce(y, y_hat):
    return - np.mean(y.reshape(y_hat.shape)*np.log(y_hat) - (1 - y.reshape(y_hat.shape))*np.log(1 - y_hat))

In [106]:
def grad_bce(y, y_hat):
    return y_hat - y.reshape(y_hat.shape)

In [107]:
# clase base MLP

class MLP():
  def __init__(self, D_in, H, D_out, loss, grad_loss, activation):
    # pesos de la capa 1
    self.w1, self.b1 = np.random.normal(loc=0.0,
    scale=np.sqrt(2/(D_in+H)),
    size=(D_in, H)), np.zeros(H)
    # pesos de la capa 2
    self.w2, self.b2 = np.random.normal(loc=0.0,
    scale=np.sqrt(2/(H+D_out)),
    size=(H, D_out)), np.zeros(D_out)
    self.ws = []
    # función de pérdida y derivada
    self.loss = loss
    self.grad_loss = grad_loss
    # función de activación
    self.activation = activation

  def __call__(self, x):
    # salida de la capa 1
    self.h_pre = np.dot(x, self.w1) + self.b1
    self.h = relu(self.h_pre)
    # salida del MLP
    y_hat = np.dot(self.h, self.w2) + self.b2
    return self.activation(y_hat)

  def fit(self, X, Y, epochs = 100, lr = 0.001, batch_size=None, verbose=True, log_each=1):
    batch_size = len(X) if batch_size == None else batch_size
    batches = len(X) // batch_size
    l = []
    for e in range(1,epochs+1):
        # Mini-Batch Gradient Descent
        _l = []
        for b in range(batches):
            # batch de datos
            x = X[b*batch_size:(b+1)*batch_size]
            y = Y[b*batch_size:(b+1)*batch_size]
            # salida del perceptrón
            y_pred = self(x)
            # función de pérdida
            loss = self.loss(y, y_pred)
            _l.append(loss)
            # Backprop
            dldy = self.grad_loss(y, y_pred)
            grad_w2 = np.dot(self.h.T, dldy)
            grad_b2 = dldy.mean(axis=0)
            dldh = np.dot(dldy, self.w2.T)*reluPrime(self.h_pre)
            grad_w1 = np.dot(x.T, dldh)
            grad_b1 = dldh.mean(axis=0)
            # Update (GD)
            self.w1 = self.w1 - lr * grad_w1
            self.b1 = self.b1 - lr * grad_b1
            self.w2 = self.w2 - lr * grad_w2
            self.b2 = self.b2 - lr * grad_b2
        l.append(np.mean(_l))
        # guardamos pesos intermedios para visualización
        self.ws.append((
            self.w1.copy(),
            self.b1.copy(),
            self.w2.copy(),
            self.b2.copy()
        ))
        if verbose and not e % log_each:
            print(f'Epoch: {e}/{epochs}, Loss: {np.mean(l):.5f}')

  def predict(self, ws, x):
    w1, b1, w2, b2 = ws
    h = relu(np.dot(x, w1) + b1)
    y_hat = np.dot(h, w2) + b2
    return self.activation(y_hat)
  def evaluate(perceptron, x, t = 0.5):
    w = perceptron.ws[-1]
    x = np.c_[np.ones(len(x)), x]
    y = perceptron(w, x)
    return (y > t).astype(np.int64)

In [108]:
# MLP para clasificación binaria
class MLPBinaryClassification(MLP):
    def __init__(self, D_in, H, D_out):
        super().__init__(D_in, H, D_out, bce, grad_bce, sigmoid)

In [109]:
model = MLPBinaryClassification(D_in=13, H=3, D_out=1)
epochs, lr = 50, 0.1

# normalización datos
x_mean = np.mean(x_train)
x_std = np.std(x_train)
x_std = np.nan_to_num(x_std, nan=1.0)


# Calcula x_norm
x_norm = (x_train - x_mean) / x_std

model.fit(x, y, epochs=100, batch_size=50, log_each=10)
#model.fit(x_norm, y_train, epochs, lr, batch_size=1, log_each=10)

C:\Users\hlope\AppData\Local\Temp\ipykernel_11668\3964504945.py:3: RuntimeWarning: divide by zero encountered in log
  return - np.mean(y.reshape(y_hat.shape)*np.log(y_hat) - (1 - y.reshape(y_hat.shape))*np.log(1 - y_hat))
C:\Users\hlope\AppData\Local\Temp\ipykernel_11668\3964504945.py:3: RuntimeWarning: invalid value encountered in multiply
  return - np.mean(y.reshape(y_hat.shape)*np.log(y_hat) - (1 - y.reshape(y_hat.shape))*np.log(1 - y_hat))


Epoch: 10/100, Loss: nan
Epoch: 20/100, Loss: nan
Epoch: 30/100, Loss: nan
Epoch: 40/100, Loss: nan
Epoch: 50/100, Loss: nan
Epoch: 60/100, Loss: nan
Epoch: 70/100, Loss: nan


Epoch: 80/100, Loss: nan
Epoch: 90/100, Loss: nan
Epoch: 100/100, Loss: nan


In [110]:
w = model.ws[-1]
w

(array([[  0.19074007,   0.90424297,   0.16686989],
        [  0.57303163,  -0.60169527,  -0.0532902 ],
        [ -1.55593072, -10.76948315,  -0.7516239 ],
        [  0.09528325,  -0.18350034,  -0.40441721],
        [ -0.17989147,  -0.06710653,  -0.14040583],
        [ -0.06356044,   0.26754762,  -0.25567709],
        [ -0.22925651,  -0.3852454 ,  -1.01022306],
        [ -0.56666488,  -0.87362061,  -2.09871584],
        [ -0.67192007,   0.35078934,   0.36264776],
        [ -0.05830167,   0.19382316,   0.52204047],
        [ -0.53904356,  -0.48237264,   0.05643711],
        [ -0.3450306 ,   0.20168615,   0.14367766],
        [  0.22663123,  -0.07063363,  -0.08430016]]),
 array([-9.00266907e-05, -2.54407778e-04, -9.13840214e-05]),
 array([[-0.79013991],
        [-0.13710218],
        [-1.45952371]]),
 array([-0.17650805]))

In [111]:
print("Primeros 5 datos de entrada de prueba (X_test):")
print(x_test[:5])

print("Primeras 5 etiquetas de prueba (y_test):")
print(y_test[:5])

Primeros 5 datos de entrada de prueba (X_test):
[[   0    3  133   11    0    3   10   42    0    0    0    2    0]
 [   0   10  692   26    0    2    8  125    0    0    0    3    0]
 [   0    6  518   22    0    3   16  117    0    0    0    4    0]
 [   0   12  777   27    0    2    7  134    0    0    0    3    0]
 [   1   17 1064   32    0    2    6  112    0    0    0    4    0]]
Primeras 5 etiquetas de prueba (y_test):
[0 0 0 0 1]


In [112]:
# nuevo punto
x_new = x_test
X_new = [6.53533935546875,619.587844574841,24.891521539971,0.109059009370733,4.27647702615029,18.9005747905553,98.6139705882353,0.293314496659804,0.086033393950794,0.530941131652007,4.47334558823529,0.981938696883038,7.45834073119875E-155]

y_pred = model.predict(w, X_new)
y_pred

array([2.58974398e-100])

In [113]:
def evaluate(perceptron, x, t = 0.5):
    w = perceptron.ws[-1]
    x = np.c_[np.ones(len(x)), x]
    y = perceptron(w, x)
    return (y > t).astype(np.int64)

In [114]:
def accuracy(y_pred, y):
    return np.sum(y_pred == y) / len(y)

In [115]:
y_pred = evaluate(perceptron, x_train)
accuracy(y_pred, y_train)

NameError: name 'perceptron' is not defined

In [116]:
y_pred = evaluate(perceptron, x_test)
accuracy(y_pred,y_test)

NameError: name 'perceptron' is not defined